In [36]:
# -------------------------------
# Grazioso Salvare Dashboard (Full-Screen, External Tab)
# -------------------------------

from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import dash_leaflet as dl
import plotly.express as px
import pandas as pd
import base64

# -------------------------------
# Load CSV Data
# -------------------------------
df = pd.read_csv("aac_shelter_outcomes.csv")

# Drop _id column if present
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Keep only dogs
df = df[df['animal_type'] == 'Dog']

# -------------------------------
# Define Rescue Type Filters Based on Assignment Table
# -------------------------------
def assign_rescue_type(row):
    breed = row['breed']
    sex = row['sex_upon_outcome']
    age = row['age_upon_outcome_in_weeks']
    
    # Water Rescue
    water_breeds = ['Labrador Retriever Mix','Chesapeake Bay Retriever','Newfoundland']
    if breed in water_breeds and sex == 'Intact Female' and 26 <= age <= 156:
        return 'Water Rescue'
    
    # Mountain/Wilderness Rescue
    mountain_breeds = ['German Shepherd','Alaskan Malamute','Old English Sheepdog','Siberian Husky','Rottweiler']
    if breed in mountain_breeds and sex == 'Intact Male' and 26 <= age <= 156:
        return 'Mountain or Wilderness Rescue'
    
    # Disaster / Individual Tracking
    disaster_breeds = ['Doberman Pinscher','German Shepherd','Golden Retriever','Bloodhound','Rottweiler']
    if breed in disaster_breeds and sex == 'Intact Male' and 20 <= age <= 300:
        return 'Disaster or Individual Tracking'
    
    return 'Other'

df['rescue_type'] = df.apply(assign_rescue_type, axis=1)

# -------------------------------
# Initialize App
# -------------------------------
app = JupyterDash(__name__)

# Encode Logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# -------------------------------
# Layout
# -------------------------------
app.layout = html.Div([
    # Header with Logo & Author
    html.Div([
        html.A(
            html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'height':'80px'}),
            href='https://www.snhu.edu', target='_blank'
        ),
        html.Center(html.B(html.H1('Grazioso Salvare Dashboard'))),
        html.P("Developed by Seth Richey")
    ], style={'textAlign': 'center'}),

    html.Hr(),

    # Rescue Type Filter Dropdown
    dcc.Dropdown(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='Reset',
        clearable=False,
        style={'width':'50%','margin':'auto'}
    ),

    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        row_selectable='single',
        sort_action='native',
        filter_action='native',
        style_table={'overflowX': 'auto', 'width': '100%'},
        style_cell={'textAlign': 'left', 'minWidth': '120px', 'width': 'auto', 'maxWidth': '300px'}
    ),

    html.Br(),
    html.Hr(),

    # Charts and Map
    html.Div(style={'display': 'flex', 'height': '80vh'}, children=[
        html.Div(id='graph-id', style={'flex': 1, 'height': '100%'}),
        html.Div(id='map-id', style={'flex': 1, 'height': '100%'})
    ])
],
style={'width': '100%', 'height': '100vh', 'margin': '0', 'padding': '0'}
)

# -------------------------------
# Callbacks
# -------------------------------

# Filter table by rescue type
@app.callback(
    Output('datatable-id','data'),
    Input('filter-type','value')
)
def update_table(filter_type):
    if filter_type == 'Reset':
        return df.to_dict('records')
    filtered_df = df[df['rescue_type'] == filter_type]
    return filtered_df.to_dict('records')

# Update Pie Chart based on filtered table
@app.callback(
    Output('graph-id', "children"),
    Input('datatable-id', "derived_virtual_data")
)
def update_pie(viewData):
    filtered_df = pd.DataFrame(viewData) if viewData else df
    fig = px.pie(filtered_df, names='breed', title='Dog Breeds Distribution')
    return [dcc.Graph(figure=fig)]

# Highlight selected DataTable columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    Input('datatable-id', 'selected_columns')
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{'if': {'column_id': i}, 'background_color': '#D2F3FF'} for i in selected_columns]

# Update Map based on selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_rows):
    dff = pd.DataFrame(viewData) if viewData else df
    if dff.empty:
        return []
    row = selected_rows[0] if selected_rows else 0
    lat, lon = dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']
    return [
        dl.Map(style={'width': '100%', 'height': '100%'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[row]['breed']),
                dl.Popup([
                    html.H4("Animal Info"),
                    html.P(f"Name: {dff.iloc[row]['name']}"),
                    html.P(f"Outcome: {dff.iloc[row]['outcome_type']}")
                ])
            ])
        ])
    ]

# -------------------------------
# Run App in external tab
# -------------------------------
app.run_server(mode='external', debug=True)


Dash app running on http://127.0.0.1:17578/
